# All Agencies

* using a sample of data from May 4th with all operators

In [ ]:
import utils
import shared_utils
from dla_utils import _dla_utils as dla_utils
from shared_utils import styleguide, altair_utils, portfolio_utils

from siuba import *
import pandas as pd

from IPython.display import display, Markdown, HTML

import altair as alt

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl

from calitp import query_sql, magics

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [30]:
may4 = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_schedule_trips_may4_sample`
''')

## alternatively 
# may4 = = query_sql(f'''
# SELECT * FROM `cal-itp-data-infra.views.gtfs_rt_vs_schedule_trips_may4_sample`
# ''')

In [4]:
may4['service_date'] = pd.to_datetime(may4['service_date'])
may4['weekday'] = pd.Series(may4.service_date).dt.day_name()    
may4['month'] =  pd.Series(may4.service_date).dt.month_name()

In [5]:
itpid_district = portfolio_utils.add_caltrans_district()
may4 = pd.merge(may4, itpid_district, on='calitp_itp_id', how='left')

In [6]:
may4.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
0,200,County Connection,0,CC:28,28,2022-05-04,2021-11-15,2099-01-01,23,21,0.91,Wednesday,May,None
1,289,RTA - SCT - Paso,0,9N,9N,2022-05-04,2021-11-18,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
2,289,RTA - SCT - Paso,0,A,A,2022-05-04,2021-11-18,2099-01-01,13,0,0.00,Wednesday,May,05 - San Luis Obispo
3,289,RTA - SCT - Paso,0,12S,12S,2022-05-04,2021-11-18,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
4,289,RTA - SCT - Paso,0,B,B,2022-05-04,2021-11-18,2099-01-01,13,0,0.00,Wednesday,May,05 - San Luis Obispo


In [7]:
may4.agency_name.nunique()

188

In [8]:
may4.calitp_itp_id.nunique()

167

In [9]:
may4>>count(_.agency_name)>>arrange(-_.n)

,agency_name,n
1,AC TRANSIT,387
173,VTA,201
142,San Francisco Municipal Transportation Agency,183
114,Orange County Transportation Authority,174
85,LA Go Bus,171
...,...,...
161,The Lift,1
177,Van Galder Coach USA,1
178,Vegas Airporter,1
180,Vermont Translines,1


In [10]:
may4>>group_by(_.calitp_itp_id)>>summarize(n=_.calitp_url_number.nunique())>>arrange(-_.n)

,calitp_itp_id,n
40,106,3
133,315,3
148,350,3
0,4,2
2,10,2
...,...,...
162,473,1
163,474,1
164,482,1
165,483,1


In [11]:
may4>>filter(_.calitp_itp_id==106)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1474,106,Emery Go-Round,1,Hollis,Hollis,2022-05-04,2021-05-13,2099-01-01,68,68,1.00,Wednesday,May,04 - Oakland
1475,106,Emery Go-Round,1,Shellmound/Powell,Shellmound/Powell,2022-05-04,2021-05-13,2099-01-01,65,65,1.00,Wednesday,May,04 - Oakland
2948,106,Emery-Go-Round,0,2094,Hollis,2022-05-04,2022-04-06,2099-01-01,68,68,1.00,Wednesday,May,04 - Oakland
2949,106,Emery Go-Round,2,4817,None,2022-05-04,2022-04-06,2099-01-01,68,0,0.00,Wednesday,May,04 - Oakland
2950,106,Emery-Go-Round,0,2080,Shellmound/Powell,2022-05-04,2022-04-06,2099-01-01,65,64,0.98,Wednesday,May,04 - Oakland
2951,106,Emery Go-Round,2,4820,None,2022-05-04,2022-04-06,2099-01-01,65,0,0.00,Wednesday,May,04 - Oakland


In [12]:
utils.agg_by_date(may4, 'num_sched', 'num_vp')

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,4,AC TRANSIT,0,2022-05-04,Wednesday,May,5378,4933,0.92
1,4,AC TRANSIT,1,2022-05-04,Wednesday,May,5378,4932,0.92
2,6,Alhambra Community Transit,0,2022-05-04,Wednesday,May,101,0,0.00
3,10,Altamont Corridor Express,0,2022-05-04,Wednesday,May,6,0,0.00
4,10,Altamont Corridor Express,1,2022-05-04,Wednesday,May,6,0,0.00
...,...,...,...,...,...,...,...,...,...
272,389,North Lake Tahoe Express - 24 hour advance res...,0,2022-05-04,Wednesday,May,19,0,0.00
273,389,Tahoe Truckee Area Regional Transit,0,2022-05-04,Wednesday,May,90,0,0.00
274,474,Tulare County Regional Transit Agency,0,2022-05-04,Wednesday,May,517,0,0.00
275,483,Tulare InterModal Express,0,2022-05-04,Wednesday,May,180,74,0.41


### By District

In [13]:
(utils.get_agg_pct(may4,
                groupings = ['caltrans_district'],
                sum_sched = 'num_sched',
                sum_vp = 'num_vp',
               ))>>select(_.caltrans_district, _.avg)

,caltrans_district,avg
0,None,NaN
1,05 - San Luis Obispo,0.27
2,07 - Los Angeles,0.67
3,04 - Oakland,0.50
4,10 - Stockton,0.68
5,03 - Marysville,0.04
6,11 - San Diego,0.96
7,02 - Redding,0.30
8,08 - San Bernardino,0.43
9,06 - Fresno,0.39


### Checking District 5 with one of the lowest percent average

In [14]:
may4>>filter(_.caltrans_district=='05 - San Luis Obispo',
            _.num_vp==0)>>count(_.agency_name)

,agency_name,n
0,A11214029,2
1,City of San Luis Obispo Transit,6
2,Guadalupe Flyer,1
3,RTA - SCT - Paso,16
4,San Benito County Express,3


In [15]:
may4>>filter( _.agency_name=='A11214029')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
71,312,A11214029,0,ROUTEA,None,2022-05-04,2021-12-03,2099-01-01,64,0,0.00,Wednesday,May,05 - San Luis Obispo
72,312,A11214029,0,ROUTEB,None,2022-05-04,2021-12-03,2099-01-01,8,0,0.00,Wednesday,May,05 - San Luis Obispo


In [16]:
may4>>filter( _.agency_name=='City of San Luis Obispo Transit')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1694,287,City of San Luis Obispo Transit,0,10867636,4A,2022-05-04,2022-01-30,2099-01-01,6,0,0.00,Wednesday,May,05 - San Luis Obispo
1695,287,City of San Luis Obispo Transit,0,10866849,3B,2022-05-04,2022-01-30,2099-01-01,12,0,0.00,Wednesday,May,05 - San Luis Obispo
1696,287,City of San Luis Obispo Transit,0,10866892,1A,2022-05-04,2022-01-30,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
1697,287,City of San Luis Obispo Transit,0,10866895,2A,2022-05-04,2022-01-30,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
1698,287,City of San Luis Obispo Transit,0,10867637,4B,2022-05-04,2022-01-30,2099-01-01,17,0,0.00,Wednesday,May,05 - San Luis Obispo
1699,287,City of San Luis Obispo Transit,0,10866897,3A,2022-05-04,2022-01-30,2099-01-01,14,0,0.00,Wednesday,May,05 - San Luis Obispo


In [17]:
may4>>filter( _.agency_name=='Guadalupe Flyer')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
3809,129,Guadalupe Flyer,0,3278,None,2022-05-04,2021-10-20,2022-05-19,22,0,0.00,Wednesday,May,05 - San Luis Obispo


In [18]:
may4>>filter( _.agency_name=='RTA - SCT - Paso')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1,289,RTA - SCT - Paso,0,9N,9N,2022-05-04,2021-11-18,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
2,289,RTA - SCT - Paso,0,A,A,2022-05-04,2021-11-18,2099-01-01,13,0,0.00,Wednesday,May,05 - San Luis Obispo
3,289,RTA - SCT - Paso,0,12S,12S,2022-05-04,2021-11-18,2099-01-01,16,0,0.00,Wednesday,May,05 - San Luis Obispo
4,289,RTA - SCT - Paso,0,B,B,2022-05-04,2021-11-18,2099-01-01,13,0,0.00,Wednesday,May,05 - San Luis Obispo
5,289,RTA - SCT - Paso,0,10S,10S,2022-05-04,2021-11-18,2099-01-01,15,0,0.00,Wednesday,May,05 - San Luis Obispo
6,289,RTA - SCT - Paso,0,10N,10N,2022-05-04,2021-11-18,2099-01-01,15,0,0.00,Wednesday,May,05 - San Luis Obispo
7,289,RTA - SCT - Paso,0,24,24,2022-05-04,2021-11-18,2099-01-01,14,0,0.00,Wednesday,May,05 - San Luis Obispo
8,289,RTA - SCT - Paso,0,21,21,2022-05-04,2021-11-18,2099-01-01,14,0,0.00,Wednesday,May,05 - San Luis Obispo
9,289,RTA - SCT - Paso,0,28,28,2022-05-04,2021-11-18,2099-01-01,14,0,0.00,Wednesday,May,05 - San Luis Obispo
10,289,RTA - SCT - Paso,0,15S,15S,2022-05-04,2021-11-18,2099-01-01,5,0,0.00,Wednesday,May,05 - San Luis Obispo


In [19]:
may4>>filter( _.agency_name=='San Benito County Express')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
113,274,San Benito County Express,0,607,None,2022-05-04,2021-12-17,2022-08-10,5,0,0.00,Wednesday,May,05 - San Luis Obispo
114,274,San Benito County Express,0,610,None,2022-05-04,2021-12-17,2022-08-10,26,0,0.00,Wednesday,May,05 - San Luis Obispo
115,274,San Benito County Express,0,606,None,2022-05-04,2021-12-17,2022-08-10,4,0,0.00,Wednesday,May,05 - San Luis Obispo


* most agencies with 0 pct ran have null vehicle positions data according to agencies.yml
* Exception: `calitp_itp_id` 312: Santa Ynez Valley Transit

Checking to see how many feeds with no vehicle position data have multiple calitp_url_numbers

In [20]:
may4>>group_by(_.calitp_itp_id, _.agency_name)>>filter(_.num_vp==0)>>summarize(n_urls = _.calitp_url_number.nunique())>>arrange(-_.n_urls)

,calitp_itp_id,agency_name,n_urls
0,4,AC TRANSIT,2
2,10,Altamont Corridor Express,2
60,56,Capitol Corridor Joint Powers Authority,2
132,218,VINE Transit,2
155,264,Rio Vista Delta Breeze,2
...,...,...,...
203,386,Yuma County Intergovernmental Public Transport...,1
204,389,North Lake Tahoe Express - 24 hour advance res...,1
205,389,Tahoe Truckee Area Regional Transit,1
206,474,Tulare County Regional Transit Agency,1


In [21]:
#which ones have multiple? might need to identify specific calitp_url_numbers
may4>>group_by(_.calitp_itp_id, _.agency_name)>>summarize(n_urls = _.calitp_url_number.nunique())>>filter(_.n_urls>1)

,calitp_itp_id,agency_name,n_urls
0,4,AC TRANSIT,2
2,10,Altamont Corridor Express,2
62,56,Capitol Corridor Joint Powers Authority,2
82,106,Emery Go-Round,2
89,110,Fairfield and Suisun Transit,2
126,194,Marin Transit,2
166,218,VINE Transit,2
180,235,Orange County Transportation Authority,2
184,246,Caltrain,2
192,264,Rio Vista Delta Breeze,2


In [31]:
may4>>filter(_.calitp_itp_id==235)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp
889,235,Orange County Transportation Authority,0,70,70,2022-05-04,2021-04-16,2099-01-01,84,0,0.00
890,235,Orange County Transportation Authority,0,167,167,2022-05-04,2021-04-16,2099-01-01,32,0,0.00
891,235,Orange County Transportation Authority,0,60,60,2022-05-04,2021-04-16,2099-01-01,99,0,0.00
892,235,Orange County Transportation Authority,0,33,33,2022-05-04,2021-04-16,2099-01-01,45,0,0.00
893,235,Orange County Transportation Authority,0,177,177,2022-05-04,2021-04-16,2099-01-01,35,0,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2817,235,Orange County Transportation Authority,1,123,123,2022-05-04,2022-04-04,2099-01-01,34,34,1.00
2818,235,Orange County Transportation Authority,1,405,405,2022-05-04,2022-04-04,2099-01-01,15,15,1.00
2819,235,Orange County Transportation Authority,0,560,560,2022-05-04,2022-04-04,2099-01-01,64,0,0.00
2820,235,Orange County Transportation Authority,0,403,403,2022-05-04,2022-04-04,2099-01-01,46,0,0.00


In [22]:
may4>>filter(_.calitp_itp_id==56)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
267,56,Capitol Corridor Joint Powers Authority,0,SF,SF,2022-05-04,2021-04-15,2099-01-01,22,0,0.00,Wednesday,May,04 - Oakland
268,56,Capitol Corridor Joint Powers Authority,0,S,Shuttle,2022-05-04,2021-04-15,2099-01-01,7,0,0.00,Wednesday,May,04 - Oakland
269,56,Capitol Corridor Joint Powers Authority,0,CC,CC,2022-05-04,2021-04-15,2099-01-01,22,0,0.00,Wednesday,May,04 - Oakland
1059,56,Capitol Corridor Joint Powers Authority,1,CC,CC,2022-05-04,2021-12-29,2099-01-01,22,0,0.00,Wednesday,May,04 - Oakland
1060,56,Capitol Corridor Joint Powers Authority,1,Shuttle,Shuttle,2022-05-04,2021-12-29,2099-01-01,7,0,0.00,Wednesday,May,04 - Oakland
1061,56,Capitol Corridor Joint Powers Authority,1,SF,SF,2022-05-04,2021-12-29,2099-01-01,22,0,0.00,Wednesday,May,04 - Oakland


* might just be some certain routes that dont have vehicle postitons

In [23]:
may4>>filter(_.num_vp==0)>>group_by(_.calitp_itp_id, _.agency_name)>>summarize(num_routes_novp = _.route_id.nunique())>>arrange(-_.num_routes_novp)

,calitp_itp_id,agency_name,num_routes_novp
169,294,VTA,70
158,273,Sacramento Regional Transit,68
168,290,SamTrans,66
165,282,San Francisco Municipal Transportation Agency,61
91,142,Orange County Transportation Authority,58
...,...,...,...
171,305,Sierra Madre Gateway Coach,1
175,315,Sonoma Marin Area Rail Transit,1
179,329,Susanville Indian Rancheria Public Transportat...,1
183,334,Susanville Indian Rancheria Public Transportat...,1


In [24]:
may4>>filter(_.num_vp==0, _.calitp_itp_id==294)>>arrange(_.calitp_url_number)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
781,294,VTA,0,71,71,2022-05-04,2021-12-27,2022-05-14,64,0,0.00,Wednesday,May,04 - Oakland
782,294,VTA,0,70,70,2022-05-04,2021-12-27,2022-05-14,129,0,0.00,Wednesday,May,04 - Oakland
783,294,VTA,0,59,59,2022-05-04,2021-12-27,2022-05-14,60,0,0.00,Wednesday,May,04 - Oakland
3587,294,VTA,0,42,42,2022-05-04,2021-09-15,2022-05-14,26,0,0.00,Wednesday,May,04 - Oakland
3588,294,VTA,0,Ornge,Orange Line,2022-05-04,2021-09-15,2022-05-14,102,0,0.00,Wednesday,May,04 - Oakland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,294,VTA,0,SCVMC,SCVMC,2022-05-04,2021-10-05,2022-05-14,17,0,0.00,Wednesday,May,04 - Oakland
3737,294,VTA,0,568,Rapid 568,2022-05-04,2021-10-05,2022-05-14,54,0,0.00,Wednesday,May,04 - Oakland
998,294,VTA,1,ACE Green,ACE Green,2022-05-04,2021-12-29,2099-01-01,8,0,0.00,Wednesday,May,04 - Oakland
1007,294,VTA,1,ACE Yellow,ACE Yellow,2022-05-04,2021-12-29,2099-01-01,8,0,0.00,Wednesday,May,04 - Oakland


In [25]:
may4>>filter(_.calitp_itp_id==294)>>group_by(_.calitp_url_number)>>summarize(n_routeid= _.route_id.nunique())

,calitp_url_number,n_routeid
0,0,67
1,1,67


In [26]:
may4>>filter(_.calitp_itp_id==294, _.calitp_url_number==1, _.route_id=='70')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1014,294,VTA,1,70,70,2022-05-04,2021-12-29,2022-06-08,129,123,0.95,Wednesday,May,04 - Oakland


In [27]:
may4>>filter(_.calitp_itp_id==294, _.calitp_url_number==0, _.route_id =='Red')

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district


In [28]:
(may4>>filter(_.calitp_itp_id==294)>>arrange(_.route_id)).head(30)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
3595,294,VTA,0,101,Express 101,2022-05-04,2021-09-15,2022-05-14,4,0,0.00,Wednesday,May,04 - Oakland
3608,294,VTA,0,102,Express 102,2022-05-04,2021-09-15,2022-05-14,10,0,0.00,Wednesday,May,04 - Oakland
3590,294,VTA,0,103,Express 103,2022-05-04,2021-09-15,2022-05-14,6,0,0.00,Wednesday,May,04 - Oakland
3643,294,VTA,0,104,Express 104,2022-05-04,2021-09-15,2022-05-14,4,0,0.00,Wednesday,May,04 - Oakland
1043,294,VTA,1,20,20,2022-05-04,2021-12-29,2099-01-01,54,54,1.00,Wednesday,May,04 - Oakland
3638,294,VTA,0,20,20,2022-05-04,2021-09-15,2022-05-14,54,0,0.00,Wednesday,May,04 - Oakland
1055,294,VTA,1,21,21,2022-05-04,2021-12-29,2099-01-01,59,59,1.00,Wednesday,May,04 - Oakland
3609,294,VTA,0,21,21,2022-05-04,2021-09-15,2022-05-14,59,0,0.00,Wednesday,May,04 - Oakland
1009,294,VTA,1,22,22,2022-05-04,2021-12-29,2099-01-01,146,144,0.99,Wednesday,May,04 - Oakland
3622,294,VTA,0,22,22,2022-05-04,2021-09-15,2022-05-14,146,0,0.00,Wednesday,May,04 - Oakland


In [29]:
overall_avg = ((may4 >>summarize(tot_sched =_.num_sched.sum(), tot_vp = _.num_vp.sum())) >> mutate(avg = _.tot_vp/_.tot_sched))
overall_avg

,tot_sched,tot_vp,avg
0,156437,88500,0.57
